#### 用协同过滤建模并进行预测

In [3]:
#载入库
from surprise import SVD
from surprise import Dataset
from surprise import evaluate,print_perf
import warnings
warnings ='ignore'

In [4]:
data = Dataset.load_builtin('ml-100k') #载入movielens数据
data.split(n_folds=3)  #k折交叉验证
algo = SVD() #尝试SVD矩阵分解
perf = evaluate(algo,data,measures=['RMSE','MAE'])  #测试效果
print(perf)

C:\ProgramData\Anaconda3\lib\site-packages\surprise\evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\surprise\dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.9491
MAE:  0.7487
------------
Fold 2
RMSE: 0.9441
MAE:  0.7442
------------
Fold 3
RMSE: 0.9434
MAE:  0.7459
------------
------------
Mean RMSE: 0.9456
Mean MAE : 0.7463
------------
------------
defaultdict(<class 'list'>, {'rmse': [0.9491131065890082, 0.9441032000022505, 0.9434427242928836], 'mae': [0.7486964804806149, 0.744189938053968, 0.7458796127937699]})


In [24]:
import os
import io
from surprise import KNNBaseline
from surprise import Dataset

#定义一个获取电影名id和id到电影名的映射函数
def read_item_names():
    file_name = (os.path.expanduser('~')+'/AppData/Local/lxss/root/.surprise_data/ml-100k/ml-100k/u.item')  #os.path.expanduser得到user路径，然后移除最后的目录部分
    rid_to_name = {}
    name_to_rid = {}
    with io.open(file_name,'r',encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rid_to_name[line[0]]=line[1]
            name_to_rid[line[1]]=line[0]
    return rid_to_name,name_to_rid

data = Dataset.load_builtin('ml-100k')

#计算相似度
trainset = data.build_full_trainset()
sim_options = {'name':'pearson_baseline','user_based':False}
algo = KNNBaseline(sim_options=sim_options)
algo.train(trainset)

#获取id和id映射
rid_to_name,name_to_rid = read_item_names()

#检索玩具总动员的inner_id
toy_story_raw_id = name_to_rid['Toy Story (1995)']
toy_story_inner_id = algo.trainset.to_inner_iid(toy_story_raw_id)

#检索距离玩具总动员最近邻id
toy_story_neighbors = algo.get_neighbors(toy_story_inner_id,k=10)

#将检索到的最近邻id转换为名字
toy_story_neighbors = (algo.trainset.to_raw_iid(inner_id) for inner_id in toy_story_neighbors)
toy_story_neighbors = (rid_to_name[rid] for rid in toy_story_neighbors)

print()
print('the 10 nearest neighbors of Toy Story are:')
for movie in toy_story_neighbors:
    print(movie)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.

the 10 nearest neighbors of Toy Story are:
Beauty and the Beast (1991)
Raiders of the Lost Ark (1981)
That Thing You Do! (1996)
Lion King, The (1994)
Craft, The (1996)
Liar Liar (1997)
Aladdin (1992)
Cool Hand Luke (1967)
Winnie the Pooh and the Blustery Day (1968)
Indiana Jones and the Last Crusade (1989)
